<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/notebooks/5_2_CatBoost_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CatBoost + Spark


* catboost-spark — это обертка для использования CatBoost в Spark-кластерах. Она позволяет обучать модели на больших данных, распределенных по узлам кластера.
* При работе с небольшими данными, то лучше использовать стандартный CatBoostClassifier без Spark.

## 1. Установка зависимостей

Используем Colab с фиксированной версией Spark 3.4 и Scala 2.12 :

In [1]:
install = False
if install:
  # Установить Spark 3.4 с Scala 2.12
  !pip install -q pyspark==3.4.0 catboost==1.2.8

  # Перезапустить среду после установки
  import os
  os.kill(os.getpid(), 9)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CatBoostExample") \
    .config("spark.jars.packages", "ai.catboost:catboost-spark_3.4_2.12:1.2.8") \
    .getOrCreate()

# Проверка загруженного артефакта
print(spark.sparkContext.getConf().get("spark.jars.packages"))

ai.catboost:catboost-spark_3.4_2.12:1.2.8


In [3]:
# Версия Spark
print("Spark version:", spark.sparkContext.version)

# Версия Scala (выведет "2.12" или "2.13")
scala_version = spark.sparkContext._jvm.scala.util.Properties.versionString()
print("Scala version:", scala_version)

Spark version: 3.4.0
Scala version: version 2.12.17


In [4]:
spark

## 2. Импорт библиотек

In [5]:
import os

os.makedirs("/content/data", exist_ok=True)
os.makedirs("/content/results", exist_ok=True)

In [6]:
# -*- coding: utf-8 -*-
import logging
import time
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from catboost_spark import CatBoostClassifier

In [7]:
PARQUET_PATH = "data/iris_data.parquet"

## 3. Настройка логгирования

In [8]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='/content/results/training.log',
    filemode='w'
)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

logger = logging.getLogger("CatBoostChinookPipeline")

## 4. Загрузка данных и преобразование в Spark DataFrame

### 4.1. Сохранение датасета в формате ```PARQUET```

In [9]:
from sklearn.datasets import load_iris
import pandas as pd

# Загрузка данных из sklearn (только для примера)
data = load_iris()
X, y = data.data, data.target

# Преобразование в Pandas DataFrame
cols = [f"{i.split()[0]}_{i.split()[1]}" for i in data.feature_names]
df = pd.DataFrame(X, columns=cols)
df["label"] = y

df.to_parquet(PARQUET_PATH, compression="gzip")

df.head(2)

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


### 3.2. Загрузка данных из ```PARQUET```

In [10]:
sdf = spark.read.parquet(PARQUET_PATH, header=True, inferSchema=True)
logger.info("Данные загружены из parquet.")
sdf.printSchema()
sdf.show(5)

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- label: long (nullable = true)

+------------+-----------+------------+-----------+-----+
|sepal_length|sepal_width|petal_length|petal_width|label|
+------------+-----------+------------+-----------+-----+
|         5.1|        3.5|         1.4|        0.2|    0|
|         4.9|        3.0|         1.4|        0.2|    0|
|         4.7|        3.2|         1.3|        0.2|    0|
|         4.6|        3.1|         1.5|        0.2|    0|
|         5.0|        3.6|         1.4|        0.2|    0|
+------------+-----------+------------+-----------+-----+
only showing top 5 rows



## 4. Стратифицированное разделение данных

In [11]:
from pyspark.sql import SparkSession, DataFrame, Window
from pyspark.sql.functions import col, rand, row_number, when
from typing import Tuple

def stratified_split(
    df: DataFrame,
    label_col: str,
    train_ratio: float = 0.6,
    val_ratio: float = 0.2,
    test_ratio: float = 0.2,
    seed: int = 42
) -> Tuple[DataFrame, DataFrame, DataFrame]:
    """
    Стратифицированное разделение данных на train/val/test с использованием оконных функций.

    Parameters:
        df (DataFrame): Исходный DataFrame.
        label_col (str): Название столбца с метками классов.
        train_ratio (float): Доля обучающей выборки.
        val_ratio (float): Доля валидационной выборки.
        test_ratio (float): Доля тестовой выборки.
        seed (int): Сид для генерации случайных чисел.

    Returns:
        tuple[DataFrame, DataFrame, DataFrame]: train_data, val_data, test_data.
    """
    # Проверка суммы долей
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-9, "Сумма долей должна быть равна 1.0"

    # Проверка наличия столбца меток
    if label_col not in df.columns:
        raise ValueError(f"Столбец '{label_col}' отсутствует в DataFrame.")

    # Добавляем случайное число для сортировки внутри групп
    df = df.withColumn("__temp_rand__", rand(seed))

    # Окно для нумерации строк внутри групп по случайному числу
    window_spec = Window.partitionBy(label_col).orderBy("__temp_rand__")

    # Добавляем номер строки в группе, начиная с 1
    df_ranked = df.withColumn("__temp_row_num__", row_number().over(window_spec))

    # Вычисляем общее количество строк в каждой группе
    group_counts = df_ranked.groupBy(label_col).count().withColumnRenamed("count", "__temp_total__")

    # Присоединяем количество строк к исходному DataFrame
    df_with_counts = df_ranked.join(group_counts, on=label_col)

    # Вычисляем пороги для train и val
    train_threshold = (col("__temp_total__") * train_ratio).cast("int")
    val_threshold = (col("__temp_total__") * (train_ratio + val_ratio)).cast("int")

    # Присваиваем метки частей
    df_partitioned = df_with_counts.withColumn(
        "__temp_partition__",
        when(col("__temp_row_num__") <= train_threshold, "train")
             .when(col("__temp_row_num__") <= val_threshold, "val")
             .otherwise("test")
    )

    # Фильтруем по меткам и убираем временные столбцы
    train_data = df_partitioned.filter(col("__temp_partition__") == "train").drop(
        "__temp_rand__", "__temp_row_num__", "__temp_total__", "__temp_partition__"
    )
    val_data = df_partitioned.filter(col("__temp_partition__") == "val").drop(
        "__temp_rand__", "__temp_row_num__", "__temp_total__", "__temp_partition__"
    )
    test_data = df_partitioned.filter(col("__temp_partition__") == "test").drop(
        "__temp_rand__", "__temp_row_num__", "__temp_total__", "__temp_partition__"
    )

    # Если val_ratio или test_ratio равны нулю, возвращаем пустой DataFrame
    if val_ratio == 0:
        val_data = df.sparkSession.createDataFrame(df.sparkSession.sparkContext.emptyRDD(), df.schema)
    if test_ratio == 0:
        test_data = df.sparkSession.createDataFrame(df.sparkSession.sparkContext.emptyRDD(), df.schema)

    return train_data, val_data, test_data

In [12]:
logger.info("Выполняется стратифицированное разделение данных")

train_data, val_data, test_data = stratified_split(sdf, "label")

logger.info(f"Разделение завершено: train={train_data.count()}, val={val_data.count()}, test={test_data.count()}")

## 5. Объединение признаков в вектор

In [13]:
logger.info("Подготовка признаков")

assembler = VectorAssembler(inputCols=[col for col in sdf.columns if col != "label"], outputCol="features")
train_assembled = assembler.transform(train_data)
val_assembled = assembler.transform(val_data)
test_assembled = assembler.transform(test_data)

## 6. Обучение модели

### 6.1 Обучение модели без кросс-валидации и оптимизации гиперпараметров

In [14]:
single_model = False
if single_model:
  logger.info("Настройка CatBoostClassifier и обучение модели")

  cb = CatBoostClassifier(
      labelCol="label",
      featuresCol="features",
      iterations=100,
      learningRate=0.1,
      depth=6
  )
  model = cb.fit(train_assembled)

  model_path = "temp_catboost_model_v1"

  logger.info(f"Сохранение модели по пути: {model_path}")
  model.save(model_path)  # Сохраняет в формате Spark ML
  logger.info("Модель успешно сохранена")

### 6.2 Обучение модели c кросс-валидацией и оптимизацией гиперпараметров

In [15]:
param_tuning = True
if param_tuning:
  logger.info("Настройка CatBoostClassifier и GridSearch")
  cb = CatBoostClassifier(
      labelCol="label",
      featuresCol="features"
  )

  # Сетка гиперпараметров
  param_grid = ParamGridBuilder() \
      .addGrid(cb.iterations, [50, 100]) \
      .addGrid(cb.learningRate, [0.03, 0.1]) \
      .addGrid(cb.depth, [4, 6]) \
      .build()

  # Оценщик
  evaluator = MulticlassClassificationEvaluator(
      labelCol="label",
      predictionCol="prediction",
      metricName="f1"
  )

  # CrossValidator
  cv = CrossValidator(
      estimator=cb,
      estimatorParamMaps=param_grid,
      evaluator=evaluator,
      numFolds=3,
      parallelism=1 # В Colab parallelism=1, a в production Для распределенного обучения
                    # лучше использовать spark-submit с отдельными процессами на каждый воркер

  )
  logger.info("Запуск кросс-валидации")
  cv_model = cv.fit(train_assembled)

  # === Лучшая модель ===
  best_model = cv_model.bestModel

  model_path = "temp_catboost_model_v2"

  logger.info(f"Сохранение модели по пути: {model_path}")
  best_model.save(model_path)  # Сохраняет в формате Spark ML
  logger.info("Модель успешно сохранена")

## 7. Загрука модели как CatBoostClassificationModel

In [16]:
from catboost_spark import CatBoostClassificationModel

loaded_model = CatBoostClassificationModel.load(model_path)

CatBoostMLReader._java_loader_class.  ai.catboost.spark.CatBoostClassificationModel


## 8. Оценка


In [17]:
logger.info("Оценка на тестовой выборке")
predictions = loaded_model.transform(test_assembled)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
metrics = {
    "Accuracy": evaluator.setMetricName("accuracy").evaluate(predictions),
    "F1-Score": evaluator.setMetricName("f1").evaluate(predictions),
    "Precision": evaluator.setMetricName("weightedPrecision").evaluate(predictions),
    "Recall": evaluator.setMetricName("weightedRecall").evaluate(predictions)
}

print(metrics)

{'Accuracy': 0.9, 'F1-Score': 0.8976982097186701, 'Precision': 0.9230769230769231, 'Recall': 0.8999999999999999}
